# P2P (Procure to Pay) Data Analysis & Visualization, Machine Learning Predictive Analytics using Julia Language

This is **Part - 3** of 3 ERP Data analysis notebooks.
- Part 1 - General Ledger, Data Science Basics
- Part 2 - General Ledger Data Analysis & Visualization
- Part 3 - P2P (Procure to Pay) Data Analysis & Visualization

**Related blogs:**
    
- [Web-scrapping, Web automation using Julia Language](https://amit-shukla.medium.com/web-scrapping-web-automation-using-julia-language-2c473db84fbc)
- Working with ODBC, ORM, XML, JSON, PDF, TXT, CSV, XLS
- Working with PDF documents, Image Scanner, OCR Reader

**Target Audience:** This notebook, is meant for ERP consultants, IT Developers, Finance, Supply chain, HR & CRM managers, executive leaders or anyone curious to implement data science concepts in ERP space.

+ **Author:** Amit Shukla
+ **Contact:** info@elishconsulting.com

In part 1, 2 of 3 series notebooks, we covered basics & details of ERP Data Finance model and learned basics of DataFrames.jl package and looked into perform detail ERP Data Analysis with visualizations.


In this part 3 notebook, we will continue to analyze Supply Chain data in aspects of Procure to Pay P2P, often referred as Buy to Pay B2P.

## adding Packages

In [1]:
using Pkg
Pkg.add("DataFrames")
Pkg.add("Dates")
Pkg.add("CategoricalArrays")
Pkg.add("Interact")
Pkg.add("WebIO")
Pkg.add("CSV")
Pkg.add("XLSX")
Pkg.add("DelimitedFiles")
Pkg.build("WebIO")
using DataFrames, Dates, Interact, CategoricalArrays, WebIO, CSV, XLSX, DelimitedFiles
Pkg.status();

      Status `C:\amit.la\WIP\GeneralLedger.jl\docs\Project.toml`
  [336ed68f] CSV v0.8.5
  [324d7699] CategoricalArrays v0.10.5
  [a93c6f00] DataFrames v1.3.2
  [e30172f5] Documenter v0.27.3
  [c601a237] Interact v0.10.4
  [0f1e0344] WebIO v0.8.17
  [fdbf4ff8] XLSX v0.7.9
  [ade2ca70] Dates
  [8bb1440f] DelimitedFiles


    Updating registry at `C:\Users\L569915\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
┌ Warning: The active manifest file at `C:\amit.la\WIP\GeneralLedger.jl\docs\Manifest.toml` has an old format that is being maintained.
│ To update to the new format run `Pkg.upgrade_manifest()` which will upgrade the format without re-resolving.
└ @ Pkg.Types C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.7\Pkg\src\manifest.jl:287
  No Changes to `C:\amit.la\WIP\GeneralLedger.jl\docs\Project.toml`
  No Changes to `C:\amit.la\WIP\GeneralLedger.jl\docs\Manifest.toml`
┌ Warning: The active manifest file is an older format with no julia version entry. Dependencies may have been resolved with a different julia version.
└ @ nothing C:\amit.la\WIP\GeneralLedger.jl\docs\Manifest.toml:0
   Resolving package versions...
  No Changes to `C:\amit.la\WIP\GeneralLedger.jl\docs\Project.toml`
  No Changes to `C:

*rest of this blog, I will assume, you have added all packages and imported in current namespace/notebook scope.*

--- 
## Supply Chain Data Model
We already covered DataFrames and ERP Finance data model in Part 1 & Part 2 notebooks, in below section, let's recreate all Supply Chain DataFrames to continue advance analytics and visualization.

#### Dimensions

- Item master, Item Attribs, Item Costing

    **UNSPSC:**  The United Nations Standard Products and Services Code® (UNSPSC®) is a global classification system of products and services.
                These codes are used to classify products and services.
    
    **GUDID:** The Global Unique Device Identification Database (GUDID) is a database administered by the FDA that will serve as a reference catalog for every device with a unique device identifier (UDI).

    **GTIN:** Global Trade Item Number (GTIN) can be used by a company to uniquely identify all of its trade items. GS1 defines trade items as products or services that are priced, ordered or invoiced at any point in the supply chain.

    **GMDN:** The Global Medical Device Nomenclature (GMDN) is a comprehensive set of terms, within a structured category hierarchy, which name and group ALL medical device products including implantables, medical equipment, consumables, and diagnostic devices.
    
- Vendor master, Vendor Attribs, Vendor Costing
    Customer/Buyer/Procurement Officer Attribs
    shipto, warehouse, storage & inventory locations

#### Transactions

-   Sales, Revenue
-   PurchaseOrder
-   MSR - Material Service
-   Voucher
-   Invoice
-   Receipt
-   Shipment
-   Travel, Expense, TimeCard
-   Accounting Lines

In [2]:
###############################
## create SUPPLY CHAIN DATA ###
###############################
# Item master, Item Attribs, Item Costing ##
#       UNSPSC, GUDID, GTIN, GMDN
############################################

##########
# UNSPSC #
##########
# UNSPSC file can be downloaded from this link https://www.ungm.org/Public/UNSPSC
xf = XLSX.readxlsx("SampleData/UNGM_UNSPSC_12-Apr-2022.xlsx")
# xf will display names of sheets and rows with data
# let's read this data in to a DataFrame

# using below command will read xlsx data into DataFrame but will not render column labels
# df = DataFrame(XLSX.readdata("UNGM_UNSPSC_12-Apr-2022.xlsx", "UNSPSC", "A1:D12988"), :auto)
dfUNSPSC = DataFrame(XLSX.readtable("SampleData/UNGM_UNSPSC_12-Apr-2022.xlsx", "UNSPSC")...)
# ... operator will splat the tuple (data, column_labels) into the constructor of DataFrame

# replace missing values with an integer 99999
replace!(dfUNSPSC."Parent key", missing => 99999)
size(dfUNSPSC)

# let's export this clean csv, we'll load this into database
CSV.write("UNSPSC.csv", dfUNSPSC)
readdir(pwd())

14-element Vector{String}:
 ".ipynb_checkpoints"
 "1-installation.ipynb"
 "FuzzyWuzzy_NLP.ipynb"
 "Image_Scanner_Reader_OCR.ipynb"
 "JuliaDataFrames-Part-1.ipynb"
 "JuliaDataFrames-Part-2.ipynb"
 "JuliaDataFrames-Part-3.ipynb"
 "SampleData"
 "UNSPSC.csv"
 "WebScrapper.ipynb"
 "Working_with_ORM-XML_JSON_Parser.ipynb"
 "images"
 "setup local machine, iPad, Andr" ⋯ 23 bytes ⋯ "ang Data Science computing.html"
 "setup local machine, iPad, Andr" ⋯ 22 bytes ⋯ "Lang Data Science computing.pdf"

In [8]:
##########
# GUDID ##
##########
# The complete list of GUDID Data Elements and descriptions can be found at this link.
# https://www.fda.gov/media/120974/download
# The complete GUDID Database (delimited version) download (250+MB)
# https://accessgudid.nlm.nih.gov/release_files/download/AccessGUDID_Delimited_Full_Release_20220401.zip
# let's extract all GUDID files in a folder
readdir("SampleData/GUDID")
# since these files are in txt (delimited) format, we'll use delimited pkg

########################
## large txt files #####
## read one at a time ##
########################

# data, header = readdlm("SampleData/GUDID/contacts.txt", '|', header=true)
# dfGUDIDcontacts = DataFrame(data, vec(header))

# data, header = readdlm("SampleData/GUDID/identifiers.txt", '|', header=true)
# dfGUDIDidentifiers = DataFrame(data, vec(header))

# data, header = readdlm("SampleData/GUDID/device.txt", '|', header=true)
# dfGUDIDdevice = DataFrame(data, vec(header))

,PrimaryDI,publicDeviceRecordKey,publicVersionStatus,deviceRecordStatus
,Any,Any,Any,Any
1,846468020071,3b9dc245-4402-48b5-aff0-8ae4187f46e5,Update,Published
2,846468020064,ad12b359-bfe3-4c0d-88da-4ee898f60009,Update,Published
3,846468020057,56f01051-273c-43a2-9451-12d6468f1e11,Update,Published
4,846468020040,d11bb977-56c4-413b-adad-f1183708e484,Update,Published
5,846468020033,f65b67b6-c828-4923-b759-313875487489,Update,Published
6,846468020026,e1a03cd1-dae1-4de3-a680-cb2f9bb1aaec,Update,Published
7,846468020019,46d86100-7844-4626-92dc-9cf400c81f25,Update,Published
8,846468020002,40dbe60d-0f3f-422f-b7cd-4699c8d187e0,Update,Published
9,846468019990,ca90fcf1-40d2-40f3-855f-e630a79ab4a0,Update,Published


In [16]:

# dfGUDIDdevice has more than 3308327 rows,
# let's split this in 6 mini files, 
# so that, it can be loaded into RDBMS easily
size(dfGUDIDdevice)
# CSV.write("dfGUDIDdevice_1.csv", dfGUDIDdevice[1:500000,:])
# CSV.write("dfGUDIDdevice_2.csv", dfGUDIDdevice[500001:1000000,:])
# CSV.write("dfGUDIDdevice_3.csv", dfGUDIDdevice[1000001:1500000,:])
# CSV.write("dfGUDIDdevice_4.csv", dfGUDIDdevice[1500001:2000000,:])
# CSV.write("dfGUDIDdevice_5.csv", dfGUDIDdevice[2000001:2500000,:])
# CSV.write("dfGUDIDdevice_6.csv", dfGUDIDdevice[2500001:3308327,:])

"dfGUDIDdevice_6.csv"

In [19]:
##########
# GTIN ###
##########

# xf = XLSX.readxlsx("SampleData/DS_GTIN_ALL.xlsx")
# xf will display names of sheets and rows with data
# let's read this data in to a DataFrame

# using below command will read xlsx data into DataFrame but will not render column labels
# df = DataFrame(XLSX.readdata("SampleData/DS_GTIN_ALL.xlsx", "Worksheet", "A14:E143403   "), :auto)
dfGTIN = DataFrame(XLSX.readtable("SampleData/DS_GTIN_ALL.xlsx", "Worksheet";first_row=14)...)
# ... operator will splat the tuple (data, column_labels) into the constructor of DataFrame

# replace missing values with an integer 99999
# replace!(dfUNSPSC."Parent key", missing => 99999)
# size(dfUNSPSC)

# let's export this clean csv, we'll load this into database
# CSV.write("UNSPSC.csv", dfUNSPSC)
# readdir(pwd())

##########
# GMDN ###
##########

## GMDN data is not available

,Product #,GTIN,Product Desc,Operating Company,Unit of Measure
,Any,Any,Any,Any,Any
1,301001030,10603295548621,ACTIS BROACH SIZE 3,Orthopaedics,EA
2,254505402-12,10603295481386,ATTUNE RP PS TRL SZ 7 12MM,Orthopaedics,EA
3,301001080,10603295548676,ACTIS BROACH SIZE 8,Orthopaedics,EA
4,254505406-12,10603295481492,ATTUNE RP PS TRL SZ 8 8MM,Orthopaedics,EA
5,L20408-13,10603295258100,BROACH CORAIL AMT 8,Orthopaedics,EA
6,254505405-12,10603295481485,ATTUNE RP PS TRL SZ 8 7MM,Orthopaedics,EA
7,L20413-13,10603295258155,BROACH CORAIL AMT 13,Orthopaedics,EA
8,254505404-12,10603295481478,ATTUNE RP PS TRL SZ 8 6MM,Orthopaedics,EA
9,L94005-13,10603295325161,CORAIL AMT NECK SEG 135D STD,Orthopaedics,EA


In [25]:
#################
# Vendor master #
#################
# unique(dfGTIN."Operating Company")
show(first(dfGUDIDdevice,5), allcols=true)

5×34 DataFrame
 Row │ PrimaryDI     publicDeviceRecordKey              publicVersionStatus  deviceRecordStatus  publicVersionNumber  publicVersionDate  devicePublishDate  deviceCommDistributionEndDate  deviceCommDistributionStatus  brandName                         versionModelNumber  catalogNumber  dunsNumber  companyName                        deviceCount  deviceDescription         DMExempt  premarketExempt  deviceHCTP  deviceKit  deviceCombinationProduct  singleUse  lotBatch  serialNumber  manufacturingDate  expirationDate  donationIdNumber  labeledContainsNRL  labeledNoNRL  MRISafetyStatus                    rx    otc  deviceSterile  sterilizationPriorToUse 
     │ Any           Any                                Any                  Any                 Any                  Any                Any                Any                            Any                           Any                               Any                 Any            Any         Any                            

--- 
## creating complete Supply Chain Data Model DataFrames
now since we got a handle of dataframe basics, let's create other chartfields/dimensions and create a complete Supply Chain DataFrame

In [81]:
## THIS IS BACKUP ##
###############################
## create SUPPLY CHAIN DATA ###
###############################
# Item master, Item Attribs, Item Costing
#       UNSPSC, GUDID, GTIN, GMDN
# vendor master, Vendor Attribs, Vendor Costing
# Item master, Item Attribs, Item Costing
# Customer/Buyer/Procurement Officer Attribs
# shipto, warehouse, storage & inventory locations
###############################
## TXNs #######################
###############################
# SALES master
# PurchaseOrder master
# MSR - Material Service Request
# Voucher master
# Invoice master
# Receipt master
# Shipment master
# travel, expense, time cards
# accounting lines
###############################

vendors = DataFrame(
    ENTITY = "Apple Inc.",
    AS_OF_DATE=Date("1900-01-01", dateformat"y-m-d"),
    ID = 11000:1000:45000,
    CLASSIFICATION=repeat([
        "OPERATING_EXPENSES","NON-OPERATING_EXPENSES", "ASSETS","LIABILITIES","NET_WORTH","STATISTICS","REVENUE"
                ], inner=5),
    CATEGORY=[
        "Travel","Payroll","non-Payroll","Allowance","Cash",
        "Facility","Supply","Services","Investment","Misc.",
        "Depreciation","Gain","Service","Retired","Fault.",
        "Receipt","Accrual","Return","Credit","ROI",
        "Cash","Funds","Invest","Transfer","Roll-over",
        "FTE","Members","Non_Members","Temp","Contractors",
        "Sales","Merchant","Service","Consulting","Subscriptions"],
    STATUS="A",
    DESCR=repeat([
    "operating expenses","non-operating expenses","assets","liability","net-worth","stats","revenue"], inner=5),
    ACCOUNT_TYPE=repeat(["E","E","A","L","N","S","R"],inner=5));

# DEPARTMENT Chartfield
deptDF = DataFrame(
    AS_OF_DATE=Date("2000-01-01", dateformat"y-m-d"), 
    ID = 1100:100:1500,
    CLASSIFICATION=["SALES","HR", "IT","BUSINESS","OTHERS"],
    CATEGORY=["sales","human_resource","IT_Staff","business","others"],
    STATUS="A",
    DESCR=[
    "Sales & Marketing","Human Resource","Infomration Technology","Business leaders","other temp"
        ],
    DEPT_TYPE=["S","H","I","B","O"]);

# LOCATION Chartfield
locationDF = DataFrame(
    AS_OF_DATE=Date("2000-01-01", dateformat"y-m-d"), 
    ID = 11:1:22,
    CLASSIFICATION=repeat([
        "Region A","Region B", "Region C"], inner=4),
    CATEGORY=repeat([
        "Region A","Region B", "Region C"], inner=4),
    STATUS="A",
    DESCR=[
"Boston","New York","Philadelphia","Cleveland","Richmond",
"Atlanta","Chicago","St. Louis","Minneapolis","Kansas City",
"Dallas","San Francisco"],
    LOC_TYPE="Physical");

# creating Ledger
ledgerDF = DataFrame(
            LEDGER = String[], FISCAL_YEAR = Int[], PERIOD = Int[], ORGID = String[],
            OPER_UNIT = String[], ACCOUNT = Int[], DEPT = Int[], LOCATION = Int[],
            POSTED_TOTAL = Float64[]
            );

# create 2020 Period 1-12 Actuals Ledger 
l = "Actuals";
fy = 2020;
for p = 1:12
    for i = 1:10^5
        push!(ledgerDF, (l, fy, p, "ABC Inc.", rand(locationDF.CATEGORY),
            rand(accountsDF.ID), rand(deptDF.ID), rand(locationDF.ID), rand()*10^8))
    end
end

# create 2021 Period 1-4 Actuals Ledger 
l = "Actuals";
fy = 2021;
for p = 1:4
    for i = 1:10^5
        push!(ledgerDF, (l, fy, p, "ABC Inc.", rand(locationDF.CATEGORY),
            rand(accountsDF.ID), rand(deptDF.ID), rand(locationDF.ID), rand()*10^8))
    end
end

# create 2021 Period 1-4 Budget Ledger 
l = "Budget";
fy = 2021;
for p = 1:12
    for i = 1:10^5
        push!(ledgerDF, (l, fy, p, "ABC Inc.", rand(locationDF.CATEGORY),
            rand(accountsDF.ID), rand(deptDF.ID), rand(locationDF.ID), rand()*10^8))
    end
end

# here is ~3 million rows ledger dataframe
size(ledgerDF)

# rename dimensions columns for innerjoin
df_accounts = rename(accountsDF, :ID => :ACCOUNTS_ID, :CLASSIFICATION => :ACCOUNTS_CLASSIFICATION, 
    :CATEGORY => :ACCOUNTS_CATEGORY, :DESCR => :ACCOUNTS_DESCR);
df_dept = rename(deptDF, :ID => :DEPT_ID, :CLASSIFICATION => :DEPT_CLASSIFICATION, 
    :CATEGORY => :DEPT_CATEGORY, :DESCR => :DEPT_DESCR);
df_location = rename(locationDF, :ID => :LOCATION_ID, :CLASSIFICATION => :LOCATION_CLASSIFICATION,
    :CATEGORY => :LOCATION_CATEGORY, :DESCR => :LOCATION_DESCR);

# join Ledger accounts chartfield with accounts chartfield dataframe to pull all accounts fields
# join Ledger dept chartfield with dept chartfield dataframe to pull all dept fields
# join Ledger location chartfield with location chartfield dataframe to pull all location fields
df_ledger = innerjoin(
                innerjoin(
                    innerjoin(ledgerDF, df_accounts, on = [:ACCOUNT => :ACCOUNTS_ID], makeunique=true),
                    df_dept, on = [:DEPT => :DEPT_ID], makeunique=true), df_location,
                on = [:LOCATION => :LOCATION_ID], makeunique=true);

# note, how ledger DF has 28 columns now (inclusive of all chartfields join)
size(df_accounts),size(df_dept),size(df_location), size(ledgerDF), size(df_ledger)

function periodToQtr(x)
    if x ∈ 1:3
        return 1
    elseif x ∈ 4:6
        return 2
    elseif x ∈ 7:9
        return 3
    else return 4
    end
end

# now we will use this function to transform a new column
transform!(df_ledger, :PERIOD => ByRow(periodToQtr) => :QTR)

# let's create one more generic function, which converts a number to USD currency
function numToCurrency(x)
        return string("USD ",round(x/10^6; digits = 2), " million")
end

transform!(df_ledger, :POSTED_TOTAL => ByRow(numToCurrency) => :TOTAL)
df_ledger[1:5,["POSTED_TOTAL","TOTAL"]]
"df_ledger_size after transformation is: ", size(df_ledger)

("df_ledger_size after transformation is: ", (2800000, 30))